In [1]:
import os
import sys
import importlib

# Sobe um nível: de notebooks/ para a raiz do projeto
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

import configs
importlib.reload(configs)  # força recarregar do disco

print("configs.py em:", configs.__file__)
print("JDBC_URL:", configs.JDBC_URL)


configs.py em: c:\Users\viami\Documents\projetos\teste-tecnico-engenharia-de-dados-sicredi-pl\configs.py
JDBC_URL: jdbc:sqlserver://AMICHI;instanceName=SQLEXPRESS;databaseName=sicredi;encrypt=false;trustServerCertificate=true;


In [2]:
from configs import (
    JDBC_URL,
    JDBC_DRIVER,
    JDBC_JAR_PACKAGE,
    DB_USER,
    DB_PASSWORD,
    BRONZE_PATH,
)
print("JDBC_URL:", JDBC_URL)

JDBC_URL: jdbc:sqlserver://AMICHI;instanceName=SQLEXPRESS;databaseName=sicredi;encrypt=false;trustServerCertificate=true;


In [3]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("sicredi-bronze")
    .config("spark.jars.packages", configs.JDBC_JAR_PACKAGE)
    .getOrCreate()
)

spark


In [4]:
connection_props = {
    "user": configs.DB_USER,
    "password": configs.DB_PASSWORD,
    "driver": configs.JDBC_DRIVER,
}

def read_table(table_name: str):
    print(f"Lendo tabela {table_name} do SQL Server...")
    return spark.read.jdbc(
        url=configs.JDBC_URL,
        table=table_name,
        properties=connection_props,
    )


In [3]:
import os
import sys
import importlib
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

import configs
importlib.reload(configs)  # força recarregar do disco

<module 'configs' from 'c:\\Users\\viami\\Documents\\projetos\\teste-tecnico-engenharia-de-dados-sicredi-pl\\configs.py'>

In [4]:
import os, sys
from pyspark.sql import SparkSession
from configs import JDBC_JAR_PATH

os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

spark = (
    SparkSession.builder
        .appName("sicredi-jdbc-test")
        .master("local[1]")
        .config("spark.jars", JDBC_JAR_PATH)
        .config("spark.driver.extraClassPath", JDBC_JAR_PATH)
        .getOrCreate()
)

spark


In [5]:
spark._jvm.java.lang.Class.forName("com.microsoft.sqlserver.jdbc.SQLServerDriver")
print("Driver JDBC carregado!")


Driver JDBC carregado!


In [6]:
spark._jvm.java.lang.Class.forName("com.microsoft.sqlserver.jdbc.SQLServerDriver")


JavaObject id=o38

In [7]:
import os, sys
from pyspark.sql import SparkSession
from configs import JDBC_JAR_PATH  # r"C:\Drivers\mssql-jdbc-13.2.1.jre11.jar"

os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

spark = (
    SparkSession.builder
        .appName("sicredi-jdbc-test")
        .master("local[*]")
        .config("spark.jars", JDBC_JAR_PATH)
        .config("spark.driver.extraClassPath", JDBC_JAR_PATH)
        .config("spark.executor.extraClassPath", JDBC_JAR_PATH)
        .getOrCreate()
)

# Teste rápido do driver
spark._jvm.java.lang.Class.forName("com.microsoft.sqlserver.jdbc.SQLServerDriver")
print("Driver JDBC carregado!")


Driver JDBC carregado!


In [8]:
from configs import JDBC_URL, JDBC_DRIVER, DB_USER, DB_PASSWORD
from py4j.protocol import Py4JJavaError

props = {
    "user": DB_USER,
    "password": DB_PASSWORD,
    "driver": JDBC_DRIVER,
}

print("JDBC_URL:", JDBC_URL)

try:
    df = spark.read.jdbc(
        url=JDBC_URL,
        table="dbo.associado",
        properties=props
    )
    df.show()
except Py4JJavaError as e:
    print("Mensagem Java:")
    print(e.java_exception.getMessage())
    raise


JDBC_URL: jdbc:sqlserver://AMICHI,1433;databaseName=sicredi;encrypt=false;trustServerCertificate=true;
Mensagem Java:
The TCP/IP connection to the host AMICHI,1433, port 1433 has failed. Error: "AMICHI,1433. Verify the connection properties. Make sure that an instance of SQL Server is running on the host and accepting TCP/IP connections at the port. Make sure that TCP connections to the port are not blocked by a firewall.".


Py4JJavaError: An error occurred while calling o42.jdbc.
: com.microsoft.sqlserver.jdbc.SQLServerException: The TCP/IP connection to the host AMICHI,1433, port 1433 has failed. Error: "AMICHI,1433. Verify the connection properties. Make sure that an instance of SQL Server is running on the host and accepting TCP/IP connections at the port. Make sure that TCP connections to the port are not blocked by a firewall.".
	at com.microsoft.sqlserver.jdbc.SQLServerException.makeFromDriverError(SQLServerException.java:250)
	at com.microsoft.sqlserver.jdbc.SQLServerException.convertConnectExceptionToSQLServerException(SQLServerException.java:316)
	at com.microsoft.sqlserver.jdbc.SocketFinder.findSocket(IOBuffer.java:2611)
	at com.microsoft.sqlserver.jdbc.TDSChannel.open(IOBuffer.java:736)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.connectHelper(SQLServerConnection.java:4201)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.login(SQLServerConnection.java:3817)
	at com.microsof